<a href="https://colab.research.google.com/github/Wazhee/KAN-vs-MLP/blob/main/U_KAN_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is KAN?

## Kolmogorov-Arnold Networks
##### New class of neural networks aimed to be more less prone to overfitting and more interpretable than multi-layer perceptrons

##### In this experiment we see if KAN can truly outperform MLP in segmentation tasks. I conduct the following experiments:
- Train U-KAN (using KAN) and U-Net (using MLP) architectures on Brats2023 Brain MRI segmentation dataset
- Evaluate U-KAN and U-Net dice similarity performance
- Evaluate U-KAN and U-Nets sensitivity to sample size and number of epochs
- Evaluate U-KAN and U-Net computational load

<img src='https://drive.google.com/file/d/1uS2mrRleKRhyF_3An_qaalMLMvU5hFqZ/view?usp=drive_link' style="height:115px">

## Load U-KAN Repository

In [1]:
!git clone https://github.com/CUHK-AIM-Group/U-KAN.git

Cloning into 'U-KAN'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 193 (delta 21), reused 15 (delta 13), pack-reused 159 (from 1)
Receiving objects: 100% (193/193), 2.58 MiB | 29.30 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [3]:
!cd U-KAN
!cd Seg_UKAN && pip install -r requirements.txt

/bin/bash: line 1: cd: Seg_UKAN: No such file or directory


In [24]:
# !pip install addict==2.4.0
# !pip install dataclasses
# !pip install pandas
# !pip install pyyaml
# !pip install albumentations
# !pip install tqdm
# !pip install
# !pip install numpy
# !pip install opencv-python
# !pip install perceptual==0.1
# !pip install pillow==8.4.0
# !pip install scikit-image==0.17.2
# !pip install scipy==1.5.4
# !pip install tifffile==2020.9.3
# !pip install timm==0.3.2
# !pip install typing-extensions==4.0.0
# !pip install yapf==0.31.0

#!pip install timm
#!pip install MedPy
!pip install tensorboardX
#!pip install typing_extensions
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

  Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl.metadata (5.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.3 MB/s eta 0:00:00


In [27]:
#!python train.py --arch UKAN --dataset {dataset} --input_w {input_size} --input_h {input_size} --name {dataset}_UKAN  --data_dir [YOUR_DATA_DIR]
!python 'U-KAN/Seg_UKAN/train.py' --arch UKAN --dataset {dataset} --input_w 128 --input_h 128 --name {dataset}_UKAN  --data_dir [YOUR_DATA_DIR]

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.17 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
--------------------
name: {dataset}_UKAN
epochs: 400
batch_size: 8
dataseed: 2981
arch: UKAN
deep_supervision: False
input_channels: 3
num_classes: 1
input_w: 128
input_h: 128
input_list: [128, 160, 256]
loss: BCEDiceLoss
dataset: {dataset}
data_dir: [YOUR_DATA_DIR]
output_dir: outputs
optimizer: Adam
lr: 0.0001
momentum: 0.9
weight_decay: 0.0001
nesterov: False
kan_lr: 0.01
kan_weight_decay: 0.0001
scheduler: CosineAnnealingLR
min_lr: 1e-05
factor: 0.1
patience: 2
milestones: 1,2
gamma: 0.6666666666666666
early_stopping: -1
cfg: None
num_workers: 4
no_kan: False
--------------------
Traceback (most recent call last):
  File "/content/U-KAN/Seg_UKAN/train.py", line 448, 